## Categorical 데이터
- Categorical형을 활용하여 pandas 메모리 사용량을 주이고 성능을 개선 가능.
- Pandas는 정수 기반의 범주형 데이터를 표현(또는 인코딩) 할 수 있는 Categorical 형이 존재한다.

In [1]:
import pandas as pd
import numpy as np

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

fruits = ['apple','orange','apple','apple']*2

N=len(fruits)

df = pd.DataFrame({'fruits':fruits,
                   'basket_id':np.arange(N),
                   'count':np.random.randint(3,15,size=N),
                   'weight':np.random.uniform(0,4,size=N)},
                 columns = ['basket_id','fruits','count','weight'])

df

,basket_id,fruits,count,weight
0,0,apple,14,1.129549
1,1,orange,13,2.529322
2,2,apple,7,2.109484
3,3,apple,5,3.968178
4,4,apple,10,2.713906
5,5,orange,4,3.187565
6,6,apple,3,1.590080
7,7,apple,12,2.807608


## 범주형 데이터로 변경 astype('category')
- **Categorical 객체는 categories와 codes 속성을 가진다**
    - categories : 범주 unique 형태
    - codes : 범주 정수형 라벨
- 범주형 으로 변경시 특정 순서를 보장하지 않음

In [2]:
fruit_cat = df['fruits'].astype('category')
fruit_cat

print('--- categories ---') # 범주 unique 형태
fruit_cat.values.categories

print('--- codes ---') # 범주 정수형 라벨
fruit_cat.values.codes 


0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruits, dtype: category
Categories (2, object): [apple, orange]

--- categories ---


Index(['apple', 'orange'], dtype='object')

--- codes ---


array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

## Categorical 연산
- groupby 같은 일부 pandas 함수는 범주형 데이터에 사용할 때 더 나은 성능을 보여준다.

In [3]:
# 랜덤 수 1000개 생성
draws = np.random.randn(1000)

# 사분위로 나눔
bins = pd.qcut(draws,4, labels=['Q1','Q2','Q3','Q4'])
bins

# categorical 형 codes확인
bins.codes[:10]

[Q3, Q2, Q1, Q4, Q1, ..., Q3, Q1, Q4, Q2, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

array([2, 1, 0, 3, 0, 3, 0, 2, 2, 1], dtype=int8)

## 메모리 성능개선 확인
- 성능 개선 : 범주형으로 변환한 DataFrame의 컬럼은 메모리를 훨씬 작게 사용한다.

In [4]:
N = 1000000

draw = pd.Series(np.random.randn(N))
labels = pd.Series(['foo','bar','baz','qux'] * (N//4))

# category 형
categories = labels.astype('category')

# 일반 Series형과 categories 형 메모리 비교
print('label 메모리 : ', labels.memory_usage())
print('categories 형 메모리 : ', categories.memory_usage())

label 메모리 :  8000128
categories 형 메모리 :  1000320


## Categorical 속성, 메서드
- cat.codes
- cat.cateogries
- cat.set_cateogries (카테고리 형 변경 (없는 데이터라도 카테고리 생성하기)
- cat.remove_unused_categories (필요없는 카테고리 제거)

In [5]:
# category 형 만들기
s = pd.Series(['a','b','c','d']*2)
cat_s = s.astype('category')
cat_s

# category 형 cat에 접근하여 codes접근
cat_s.cat.codes

# category 형 cat에 접근하여 categories 접근
cat_s.cat.categories

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

Index(['a', 'b', 'c', 'd'], dtype='object')

In [7]:
## set_categories - 카테고리 형 변경 (없는 데이터라도 카테고리 생성하기)

actual_categories = ['a','b','c','d','e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [8]:
## remove_unused_categories (필요없는 카테고리 제거)
cat_s3 = cat_s[cat_s.isin(['a','b'])]
cat_s3

# 안쓰는 카테고리 제거
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

In [9]:
## category 형을 dummy 이용해서 one hot 으로도 변경 가능
cat_s = pd.Series(['a','b','c','d']*2, dtype='category')
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## apply와 유사한 transform 메서드
- 그룹 형태로 브로드캐스트할 수 있는 스칼라값을 생성해야 함.
- 입력 그룹과 같은 형태의 객체를 반환. 

In [15]:
df = pd.DataFrame({'key':['a','b','c']*2,
                   'value':[10,3,7,9,1,3]})
df

# key에 따른 그룹의 평균구하기
g = df.groupby('key').value
g.mean()

,key,value
0,a,10
1,b,3
2,c,7
3,a,9
4,b,1
5,c,3


key
a    9.5
b    2.0
c    5.0
Name: value, dtype: float64

In [24]:
# transform
g.transform(lambda x:x.max()-x.min())
print('-- transform max()-min() --')

# 내장 요약함수에 대해서는 문자열 그룹 연산 이름 넘기기 가능
g.transform('mean')
print('-- 내장요약함수 mean --')

# 각 그룹 2곱하기
g.transform(lambda x:x*2)
print('-- 각 그룹 2곱하기 --')

# 그룹별 내림차순 순위 
g.transform(lambda x:x.rank(ascending=False))
print('-- 그룹별 내림차순 순위 --')

# normalize
def normalize(x):
    return (x-x.mean() / x.std())
g.transform(normalize)
print('-- 정규화 --')


0    1
1    2
2    4
3    1
4    2
5    4
Name: value, dtype: int64

-- transform max()-min() --


0    9.5
1    2.0
2    5.0
3    9.5
4    2.0
5    5.0
Name: value, dtype: float64

-- 내장요약함수 mean --


0    20
1     6
2    14
3    18
4     2
5     6
Name: value, dtype: int64

-- 각 그룹 2곱하기 --


0    1
1    1
2    1
3    2
4    2
5    2
Name: value, dtype: int64

-- 그룹별 내림차순 순위 --


0   -3.435029
1    1.585786
2    5.232233
3   -4.435029
4   -0.414214
5    1.232233
Name: value, dtype: float64

-- 정규화 --
